# 使用Pinecone对GPT-4进行检索增强#### 修复产生幻觉的LLMs在这个笔记本中，我们将学习如何从Pinecone查询与我们的查询相关的上下文，并将这些上下文传递给GPT-4模型，以生成一个由真实数据源支持的答案。GPT-4相比于之前的OpenAI完成模型有了很大的进步。它还专门使用`ChatCompletion`端点，因此我们必须以稍微不同的方式使用它。然而，这个模型的强大之处使得这种改变是值得的，特别是当与Pinecone向量数据库等外部知识库相结合时。本笔记本所需的安装包括：

In [1]:
!pip install -qU bs4 tiktoken openai langchain pinecone-client[grpc]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.0/396.0 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 准备数据

在这个例子中，我们将从[langchain.readthedocs.io/](https://langchain.readthedocs.io/latest/en/)下载LangChain文档。我们可以这样获取该网站上所有`.html`文件：

In [56]:
!wget -r -A.html -P rtdocs https://python.langchain.com/en/latest/

<Response [200]>

这将所有HTML下载到`rtdocs`目录中。现在我们可以使用LangChain本身来处理这些文档。我们可以这样使用`ReadTheDocsLoader`：

In [57]:
from langchain.document_loaders import ReadTheDocsLoaderloader = ReadTheDocsLoader('rtdocs')docs = loader.load()len(docs)

.rst .pdf Welcome to LangChain Contents Getting Started Modules Use Cases Reference Docs LangChain Ecosystem Additional Resources Welcome to LangChain# Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. But using these LLMs in isolation is often not enough to create a truly powerful app - the real power comes when you are able to combine them with other sources of computation or knowledge. This library is aimed at assisting in the development of those types of applications. Common examples of these types of applications include: ❓ Question Answering over specific documents Documentation End-to-end Example: Question Answering over Notion Database 💬 Chatbots Documentation End-to-end Example: Chat-LangChain 🤖 Agents Documentation End-to-end Example: GPT+WolframAlpha Getting Started# Checkout the below guide for a walkthrough of how to get started using LangChain to create an Language Model app

这样我们就得到了数百个处理过的文档页面。让我们来看看每个页面包含的格式：

In [ ]:
docs[0]

我们可以这样访问纯文本页面内容：

In [58]:
print(docs[0].page_content)

In [ ]:
print(docs[5].page_content)

我们还可以找到每个文档的来源：

In [ ]:
docs[5].metadata['source'].replace('rtdocs/', 'https://')

我们可以使用这些来创建我们的`data`列表：

In [ ]:
data = []for doc in docs:    data.append({        'url': doc.metadata['source'].replace('rtdocs/', 'https://'),        'text': doc.page_content    })

In [60]:
data[3]

{'url': 'https://langchain.readthedocs.io/en/latest/modules/memory/types/entity_summary_memory.html',
 'text': '.ipynb .pdf Entity Memory Contents Using in a chain Inspecting the memory store Entity Memory# This notebook shows how to work with a memory module that remembers things about specific entities. It extracts information on entities (using LLMs) and builds up its knowledge about that entity over time (also using LLMs). Let’s first walk through using this functionality. from langchain.llms import OpenAI from langchain.memory import ConversationEntityMemory llm = OpenAI(temperature=0) memory = ConversationEntityMemory(llm=llm) _input = {"input": "Deven & Sam are working on a hackathon project"} memory.load_memory_variables(_input) memory.save_context( _input, {"ouput": " That sounds like a great project! What kind of project are they working on?"} ) memory.load_memory_variables({"input": \'who is Sam\'}) {\'history\': \'Human: Deven & Sam are working on a hackathon project\\nAI: 

虽然看起来有点丑，但现在已经足够好了。让我们看看如何处理所有这些内容。我们将把所有内容分成大约400个标记的块，我们可以很容易地使用`langchain`和`tiktoken`来实现这一点：

In [61]:
import tiktokentokenizer = tiktoken.get_encoding('p50k_base')# 创建长度函数def tiktoken_len(text):    tokens = tokenizer.encode(        text,        disallowed_special=()    )    return len(tokens)

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplittertext_splitter = RecursiveCharacterTextSplitter(    chunk_size=400,    chunk_overlap=20,    length_function=tiktoken_len,    separators=["\n\n", "\n", " ", ""])

使用这种方法将`data`处理成更多的块。

In [63]:
from uuid import uuid4from tqdm.auto import tqdmchunks = []for idx, record in enumerate(tqdm(data)):    texts = text_splitter.split_text(record['text'])    chunks.extend([{        'id': str(uuid4()),        'text': texts[i],        'chunk': i,        'url': record['url']    } for i in range(len(texts))])

  0%|          | 0/231 [00:00<?, ?it/s]

我们的块已经准备好了，现在我们开始嵌入和索引所有内容。

## 初始化嵌入模型我们使用`text-embedding-3-small`作为嵌入模型。我们可以这样嵌入文本：

In [66]:
import openai# 初始化OpenAI API密钥openai.api_key = "sk-..."embed_model = "text-embedding-3-small"res = openai.Embedding.create(    input=[        "Sample document text goes here",        "there will be several phrases in each batch"    ], engine=embed_model)

在响应`res`中，我们将在`'data'`字段中找到一个包含我们新嵌入向量的类似JSON的对象。

In [67]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

在`'data'`中，我们会找到两条记录，分别对应我们刚刚嵌入的两个句子。每个向量嵌入包含`1536`个维度（即`text-embedding-3-small`模型的输出维度）。

In [68]:
len(res['data'])

2

In [69]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

我们将把相同的嵌入逻辑应用到我们刚刚抓取的langchain文档数据集中。但在此之前，我们必须创建一个存储嵌入的位置。

## 初始化索引

现在我们需要一个地方来存储这些嵌入向量，并通过它们进行高效的向量搜索。为此，我们使用Pinecone，可以在[此处](https://app.pinecone.io/)获取免费的API密钥，并在下面输入该密钥，我们将初始化与Pinecone的连接并创建一个新的索引。

In [70]:
import pineconeindex_name = 'gpt-4-langchain-docs'# 初始化与 Pinecone 的连接pinecone.init(    api_key="PINECONE_API_KEY",  # app.pinecone.io（控制台）    environment="PINECONE_ENVIRONMENT"  # 在控制台中，API密钥的旁边)# check if index already exists (it shouldn't if this is first time)if index_name not in pinecone.list_indexes():    # if does not exist, create index    pinecone.create_index(        index_name,        dimension=len(res['data'][0]['embedding']),        metric='dotproduct'    )# 连接到索引index = pinecone.GRPCIndex(index_name)# 查看索引统计信息index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

我们可以看到索引目前是空的，`total_vector_count`为`0`。我们可以开始使用OpenAI `text-embedding-3-small`构建的嵌入来填充它，就像这样：

In [71]:
from tqdm.auto import tqdmimport datetimefrom time import sleepbatch_size = 100  # 我们一次创建和插入多少个嵌入？for i in tqdm(range(0, len(chunks), batch_size)):    # 查找批处理结束位置    i_end = min(len(chunks), i+batch_size)    meta_batch = chunks[i:i_end]    # 获取ID    ids_batch = [x['id'] for x in meta_batch]    # 获取待编码文本    texts = [x['text'] for x in meta_batch]    # 创建嵌入（添加了try-except以避免RateLimitError）    try:        res = openai.Embedding.create(input=texts, engine=embed_model)    except:        done = False        while not done:            sleep(5)            try:                res = openai.Embedding.create(input=texts, engine=embed_model)                done = True            except:                pass    embeds = [record['embedding'] for record in res['data']]    # 清理元数据    meta_batch = [{        'text': x['text'],        'chunk': x['chunk'],        'url': x['url']    } for x in meta_batch]    to_upsert = list(zip(ids_batch, embeds, meta_batch))    # 插入或更新到 Pinecone    index.upsert(vectors=to_upsert)

  0%|          | 0/12 [00:00<?, ?it/s]

现在我们已经将所有的langchain文档添加到索引中了。有了这个，我们可以继续进行检索，然后使用GPT-4进行答案生成。

## 检索

要搜索我们的文档，首先需要创建一个查询向量 `xq`。使用 `xq`，我们将从LangChain文档中检索出最相关的片段，就像这样：

In [83]:
query = "how do I use the LLMChain in LangChain?"res = openai.Embedding.create(    input=[query],    engine=embed_model)# 从Pinecone检索xq = res['data'][0]['embedding']# 获取相关上下文（包括问题）res = index.query(xq, top_k=5, include_metadata=True)

In [84]:
res

{'matches': [{'id': '1fec660b-9937-4f7e-9692-280c8cc7ce0d',
              'metadata': {'chunk': 0.0,
                           'text': '.rst .pdf Chains Chains# Using an LLM in '
                                   'isolation is fine for some simple '
                                   'applications, but many more complex ones '
                                   'require chaining LLMs - either with each '
                                   'other or with other experts. LangChain '
                                   'provides a standard interface for Chains, '
                                   'as well as some common implementations of '
                                   'chains for ease of use. The following '
                                   'sections of documentation are provided: '
                                   'Getting Started: A getting started guide '
                                   'for chains, to get you up and running '
                                   'quickly.

检索完成后，我们将继续将这些内容输入到GPT-4中，以生成答案。

## 检索增强生成

GPT-4目前通过OpenAI的`ChatCompletions`端点进行访问。为了将我们检索到的信息添加到模型中，我们需要将其传递到我们的用户提示中，*并与*我们的原始查询一起。我们可以这样做：

In [85]:
# 获取检索到的文本列表contexts = [item['metadata']['text'] for item in res['matches']]augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query

In [87]:
print(augmented_query)

.rst .pdf Chains Chains# Using an LLM in isolation is fine for some simple applications, but many more complex ones require chaining LLMs - either with each other or with other experts. LangChain provides a standard interface for Chains, as well as some common implementations of chains for ease of use. The following sections of documentation are provided: Getting Started: A getting started guide for chains, to get you up and running quickly. Key Concepts: A conceptual guide going over the various concepts related to chains. How-To Guides: A collection of how-to guides. These highlight how to use various types of chains. Reference: API reference documentation for all Chain classes. previous Vector DB Text Generation next Getting Started By Harrison Chase © Copyright 2022, Harrison Chase. Last updated on Mar 15, 2023.

---

.rst .pdf LLMs LLMs# Large Language Models (LLMs) are a core component of LangChain. LangChain is not a provider of LLMs, but rather provides a standard interface thr

现在我们提出问题：

In [88]:
# system message to 'prime' the modelprimer = f"""You are Q&A bot. A highly intelligent system that answersuser questions based on the information provided by the user aboveeach question. If the information can not be found in the informationprovided by the user you truthfully say "I don't know"."""res = openai.ChatCompletion.create(    model="gpt-4",    messages=[        {"role": "system", "content": primer},        {"role": "user", "content": augmented_query}    ])

为了漂亮地显示这个回复，我们将以markdown格式显示它。

In [89]:
from IPython.display import Markdowndisplay(Markdown(res['choices'][0]['message']['content']))

To use the LLMChain in LangChain, follow these steps:

1. Import the necessary classes:
```python
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
```

2. Create an instance of the LLM and set the configuration options:
```python
llm = OpenAI(temperature=0.9)
```

3. Create a PromptTemplate instance with the input variables and the template:
```python
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good product for {product}?",
)
```

4. Create an LLMChain instance by passing the LLM and PromptTemplate instances:
```python
llm_chain = LLMChain(llm=llm, prompt_template=prompt)
```

5. Run the LLMChain with user input:
```python
response = llm_chain.run({"product": "software development"})
```

6. Access the generated response:
```python
generated_text = response["generated_text"]
```

In this example, the LLMChain is used to generate a response by passing through the user input and formatting it using the prompt template. The response is then obtained from the LLM instance (in this case, OpenAI), and the generated text can be accessed from the response dictionary.

让我们将其与未增强的查询进行比较...

In [90]:
res = openai.ChatCompletion.create(    model="gpt-4",    messages=[        {"role": "system", "content": primer},        {"role": "user", "content": query}    ])display(Markdown(res['choices'][0]['message']['content']))

I don't know.

如果我们去掉`primer`中的`"I don't know"`部分呢？

In [91]:
res = openai.ChatCompletion.create(    model="gpt-4",    messages=[        {"role": "system", "content": "You are Q&A bot. A highly intelligent system that answers user questions"},        {"role": "user", "content": query}    ])display(Markdown(res['choices'][0]['message']['content']))

LangChain hasn't provided any public documentation on LLMChain, nor is there a known technology called LLMChain in their library. To better assist you, please provide more information or context about LLMChain and LangChain.

Meanwhile, if you are referring to LangChain, a blockchain-based decentralized AI language model, you can start by visiting their official website (if they have one), exploring their available resources, such as documentation and tutorials, and following any instructions on setting up their technology.

If you are looking for help with a specific language chain or model in natural language processing, consider rephrasing your question to provide more accurate information or visit relevant resources like GPT-3 or other NLP-related documentation.